* Given a string s, return the longest palindromic substring in s. (最長對稱字串)
* Example 1:
    ```
    Input: s = "babad"
    Output: "bab"
    Explanation: "aba" is also a valid answer.
    ```
* Example 2:
    ```
    Input: s = "cbbd"
    Output: "bb"
    ```

### 一開始的直覺思路(錯):
* 直覺來講對稱字會想用雙指針硬檢查；對稱字也不存在奇偶數問題。
* case感覺只有三種:
    1. 單一的單字(edge case) > 直接return
    2. 偶數長度的 > abba
    3. 奇數長度的 > abcba


#### Solution 1: 中心展開法
* 當尋找最長的回文子字串時，需要考慮兩種情況，這就是為什麼會有 p1 和 p2 兩個回文。奇數長度的回文子字串（Odd-length palindromes）: 在這種情況下，回文子字串有一個確定的中心字元，其他字元圍繞著這個中心字元對稱。例如，"aba" 或 "racecar"。這種情況對應於 p2 = self.get_palindrome(s, i, i)，其中 i 是中心字元的索引。偶數長度的回文子字串（Even-length palindromes）: 在這種情況下，回文子字串沒有一個確定的中心字元，而是有一對中心字元。其他字元圍繞著這對中心字元對稱。例如，"abba" 或 "civic"。這種情況對應於 p1 = self.get_palindrome(s, i, i+1)，其中 i 和 i+1 是中心字元對的索引。
* 時間複雜度: O(n^2)
* 空間複雜度: O(1)

* code細節:
```
在 get_palindrome 函數中，我們使用 while 迴圈從中心點 (l, r) 開始向外擴展，檢查是否可以形成更長的回文子字串。當迴圈結束時，(l, r) 的位置會是已經超過了回文子字串邊界的第一對不匹配的字元。

為了更清楚地說明，讓我們考慮一個例子：字串 s = "abba"，並且我們從中心點 (l=1, r=2) （即第二個和第三個字元，都是 'b'）開始擴展。

初始：l=1, r=2，子字串是 "bb"。
第一次迭代：l=0, r=3，子字串是 "abba"。
迴圈結束：l=-1, r=4，這時候已經超過了字串的邊界。
注意，在迴圈結束時，l 和 r 的位置是不包含在最後的回文子字串中的。所以，我們需要使用 s[l+1:r] 來獲得真正的回文子字串，這就是為什麼是 l+1 而不是 l。在這個例子中，s[-1+1:4] = s[0:4] = "abba"，這就是我們想要的回文子字串。
```

In [9]:
class Solution:
    def longestPalindrome(self, s: str) -> str:
        # 初始化一個空字串 p 來儲存最長的回文子字串
        p = ''
        
        # 遍歷字串 s 的每一個字元
        for i in range(len(s)):
            # 找出以 i 和 i+1 為中心的最長回文子字串
            p1 = self.get_palindrome(s, i, i+1)
            
            # 找出以 i 為中心的最長回文子字串
            p2 = self.get_palindrome(s, i, i)
            
            # 比較目前找到的最長回文子字串與 p1 和 p2，然後更新 p
            p = max([p, p1, p2], key=len)
        
        # 回傳最長的回文子字串
        return p

    def get_palindrome(self, s: str, l: int, r: int) -> str:
        '''從i開始往左右找, 找到最長回文後回傳'''
        # 使用 while 迴圈來從中心 (l, r) 展開
        # r不是<=len(s)原因是因為range(len(s))是從0開始, len(s)會比range(len(s))溢出多一格
        while l >= 0 and r < len(s) and s[l] == s[r]: 
            # 若滿足擴展條件則擴展
            # 我更新的是next iterate的初始位置，所以最後一定會溢出
            l -= 1
            r += 1
            print(f'l:{l} / r:{r}')
        
        # 回傳找到的回文子字串
        # 因為更新方式會溢出，所以需要l+1
        return s[l+1:r]

In [10]:
sol = Solution()
sol.longestPalindrome('aaaabbbsbbbaaedded')

l:-1 / r:2
l:-1 / r:1
l:0 / r:3
l:-1 / r:4
l:0 / r:2
l:-1 / r:3
l:1 / r:4
l:1 / r:3
l:0 / r:4
l:2 / r:4
l:3 / r:6
l:3 / r:5
l:4 / r:7
l:4 / r:6
l:3 / r:7
l:5 / r:7
l:6 / r:8
l:5 / r:9
l:4 / r:10
l:3 / r:11
l:2 / r:12
l:1 / r:13
l:7 / r:10
l:7 / r:9
l:8 / r:11
l:8 / r:10
l:7 / r:11
l:9 / r:11
l:10 / r:13
l:10 / r:12
l:11 / r:13
l:12 / r:14
l:13 / r:16
l:12 / r:17
l:13 / r:15
l:14 / r:16
l:15 / r:17
l:14 / r:18
l:16 / r:18


'aabbbsbbbaa'

#### Solution 2: DP

In [ ]:
class Solution:
    def longestPalindrome(self, s: str) -> str:
        # 初始化動態規劃表格 dp，dp[i][j] 將用來存儲 s[j:i+1] 是否是回文
        dp = [[False] * len(s) for _ in range(len(s))]
        
        # 初始化一個空字串 longest，用於儲存最長回文子字串
        longest = ''
        
        # 外層 for 迴圈遍歷字串 s 的每一個字元
        for i in range(len(s)):
            # 內層 for 迴圈從 0 遍歷到 i
            for j in range(i + 1):
                # 檢查 s[i] 和 s[j] 是否相同，並檢查子字串長度是否小於等於 3 或 dp[i-1][j+1] 是否為 True
                if s[i] == s[j] and ((i + 1 - j) <= 3 or dp[i - 1][j + 1]):
                    # 如果條件滿足，則 s[j:i+1] 是一個回文子字串
                    dp[i][j] = True
                    
                    # 檢查這個回文子字串是否比目前最長的回文子字串還要長
                    if i + 1 - j > len(longest):
                        longest = s[j:i + 1]
        
        # 回傳找到的最長回文子字串
        return longest

#### Brute-Force
* 時間複雜度: O(n^3)
* 空間複雜度: O(1)

In [ ]:
class Solution:
    def longestPalindrome(self, s: str) -> str:
        # 初始化一個空字串用於儲存最長的回文子字串
        longest = ""
        
        # 遍歷所有可能的子字串
        for i in range(len(s)):
            for j in range(i + 1, len(s) + 1):
                # 獲取當前子字串
                substring = s[i:j]
                
                # 檢查子字串是否是回文
                if substring == substring[::-1]:
                    # 檢查找到的回文子字串是否比目前最長的還要長
                    if len(substring) > len(longest):
                        longest = substring
                        
        return longest
